# Domestic Load Research Data Explorer for the Curious
This notebook contains the functions that do the data processing in the DLR Data Explorer app so that curious researchers can have a more fine grained view of the dataset.

## Function and Data Setup

### Import Statements

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
from dash.dependencies import Input, Output#, State

import plotly.graph_objs as go
import plotly.offline as po
po.init_notebook_mode(connected=True)

import pandas as pd
import numpy as np
import os
import base64

import features
from support import appProfiles 

mapbox_access_token = 'pk.eyJ1Ijoic2FpbnRseXZpIiwiYSI6ImNqZHZpNXkzcjFwejkyeHBkNnp3NTkzYnQifQ.Rj_C-fOaZXZTVhTlliofMA'

### Data Import
Requires datasets to be in the /data folder

In [2]:
# Get load profile data from disk
profiles = appProfiles(1994,2014)  

# Load datasets
ids = features.loadID()

#a little bit of data wrangling
loc_summary = pd.pivot_table(ids, values = ['AnswerID'], index = ['Year','LocName','Lat','Long','Municipality','Province'],aggfunc = np.count_nonzero)
loc_summary.reset_index(inplace=True)
loc_summary.rename(columns={'AnswerID':'# households'}, inplace=True)

## Survey Locations

## Map View

In [3]:
def create_map(ids_df):
 
    georef = pd.pivot_table(ids_df, values = ['AnswerID'], index = ['Year','LocName','Lat','Long','Municipality','Province'],aggfunc = np.count_nonzero)
    georef.reset_index(inplace=True)
    georef.rename(columns={'AnswerID':'# households'}, inplace=True)
                           
    traces = []
    for y in range(georef.Year.min(), georef.Year.max()+1):
        lat = georef.loc[(georef.Year==y), 'Lat']
        lon = georef.loc[(georef.Year==y), 'Long']
        text = georef.loc[(georef.Year==y), '# households'].astype(str) + ' household surveys</br>'+ georef.loc[(georef.Year==y), 'LocName'] + ', ' + georef.loc[(georef.Year==y), 'Municipality']
        marker_size = georef.loc[georef.Year==y,'# households']**(1/2.5)*2.7
        marker_size.replace([0,1,2,3,4, 5], 6, inplace=True)
        trace=go.Scattermapbox(
                name=y,
                lat=lat,
                lon=lon,
                mode='markers',
                marker=go.Marker(
                    size=marker_size
                ),
                text=text,
            )
        traces.append(trace)
    figure=go.Figure(
        data=go.Data(traces),
        layout = go.Layout(
                autosize=True,
                hovermode='closest',
                mapbox=dict(
                    accesstoken=mapbox_access_token,
                    bearing=0,
                    center=dict(
                        lat=-29.1,
                        lon=25
                    ),
                    pitch=0,
                    zoom=4.32,
                    style='light'
                ),
                margin = go.Margin(
                        l = 10,
                        r = 10,
                        t = 20,
                        b = 30
                ),
                showlegend=True
            )
    )
    return po.iplot( figure)

In [4]:
create_map(ids)

### Total surveyed households by year

In [5]:
pd.DataFrame(loc_summary.groupby(['Year'])['# households'].sum())

,# households
Year,
1994,50
1995,100
1996,200
1997,336
1998,547
1999,443
2000,904
2001,717
2002,751


### Total surveyed household count by municipality

In [6]:
pd.DataFrame(loc_summary.groupby(['Province','Municipality'])['# households'].sum())

# households
Province Municipality                           
EC       Enoch Mgijima                       268
         Kouga                               336
         Mhlontlo                             38
         Mnquma                              153
         Nelson Mandela Bay                  572
FS       Mangaung                            290
         Mantsopa                            134
         Matjhabeng                            0
GP       City of Johannesburg                311
         City of Tshwane                     385
         Ekurhuleni                          517
         Emfuleni                              0
KZN      Alfred Duma                         295
         Mkhambathini                         49
         Ray Nkonyeni                        177
         The Msunduzi                        143
         Umzimkhulu                          295
         eThekwini                           810
         uMhlathuze                          787
LIM      Ephraim Mogale                       67
         Greater Tubatse/Fetakgomo           761
         Lepele-Nkumpi                       140
MP       Mbombela                            141
         Nkomazi                             130
         Thembisile                          985
NC       Joe Morolong                        251
NW       Ramotshere Moiloa                   301
         Rustenburg                          348
WC       City of Cape Town                  1464
         Stellenbosch                         50

### Surveyed household count per municipality per year (1994 - 2014)

In [7]:
loc_summary.pivot_table(aggfunc=sum,columns='Year',index=['Province','Municipality'],values='# households',fill_value='')

Year                               1994 1995 1996 1997 1998 1999 2000 2001  \
Province Municipality                                                        
EC       Enoch Mgijima                               0    0    0   61   63   
         Kouga                                                               
         Mhlontlo                                              0   38        
         Mnquma                                                              
         Nelson Mandela Bay               66        71   69   54   57   50   
FS       Mangaung                                                            
         Mantsopa                              59   75                       
         Matjhabeng                                                          
GP       City of Johannesburg                  69   73   77   35   57        
         City of Tshwane                                                50   
         Ekurhuleni                                                          
         Emfuleni                                                            
KZN      Alfred Duma                                                         
         Mkhambathini                                                        
         Ray Nkonyeni                                                   58   
         The Msunduzi                          72   71                       
         Umzimkhulu                                            0   71   83   
         eThekwini                        34         0  271  190  202   51   
         uMhlathuze                                                          
LIM      Ephraim Mogale                              0    0    0   67        
         Greater Tubatse/Fetakgomo                             0   62   57   
         Lepele-Nkumpi                                    0    0   71   69   
MP       Mbombela                                                            
         Nkomazi                                                             
         Thembisile                                                          
NC       Joe Morolong                                          0   66   54   
NW       Ramotshere Moiloa                                                   
         Rustenburg                                                          
WC       City of Cape Town                          46  130  164  152  182   
         Stellenbosch                50                                      

Year                               2002 2003 ...  2005 2006 2007 2008 2009  \
Province Municipality                        ...                             
EC       Enoch Mgijima               73   71 ...                             
         Kouga                               ...                             
         Mhlontlo                            ...                             
         Mnquma                              ...                             
         Nelson Mandela Bay          57    0 ...    60   60                  
FS       Mangaung                         77 ...                             
         Mantsopa                            ...                             
         Matjhabeng                          ...                             
GP       City of Johannesburg                ...                             
         City of Tshwane             56   58 ...     0   74    0   72   75   
         Ekurhuleni                          ...                         0   
         Emfuleni                            ...                             
KZN      Alfred Duma                         ...    78   80    0   62   75   
         Mkhambathini                 0   49 ...                             
         Ray Nkonyeni                60   59 ...                             
         The Msunduzi                        ...                             
         Umzimkhulu                  72   69 ...                             
         eThekwini   

## Socio-demographic Summaries

### Understanding Survey Data

#### Search Survey Questions for Key Words

In [55]:
features.searchQuestions('liv')

,Question,Datatype,QuestionaireID,ColumnNo
707,PeopleAreLivingAtShop,num,1000001,5
714,NumberOfPeopleLivingAtShop,num,1000001,10
791,PeopleAreLivingAtSchool,num,1000002,12
792,NumOfPeopleLivingAtSchool,num,1000002,13


#### Create Dataframe with Selected Features

In [45]:
appliances = ['fridge freezer','geyser','heater','hotplate','iron','kettle','microwave','3 plate','4 plate','tv',
 'washing machine']
other_socios = ['main switch', 'floor area', 'rooms', 'wall']
 
#load socio-demographic feature frame
sd = features.socio_demographics(appliances, other_socios)
sd.head()

,AnswerID,QuestionaireID,years_electrified,monthly_income,fridge_freezer,geyser,heater,hotplate,iron,kettle,microwave,3_plate,4_plate,tv,washing_machine,main switch,floor area,rooms,wall
0,1415,6,10.0,2900.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,60.0,50.0,5.0,3.0
1,1414,6,10.0,1450.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,60.0,50.0,5.0,3.0
2,1411,6,10.0,2550.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,60.0,50.0,5.0,3.0
3,1412,6,10.0,1300.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,60.0,50.0,6.0,3.0
4,1409,6,10.0,0.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,60.0,50.0,5.0,3.0


#### View Locations for Year

In [46]:
#Specify Survey Year
year = 2005

ids_yr = ids[ids.Year==year]
list(ids_yr.LocName.unique())

['Khayalitsha',
 'Greenturf',
 'Vlaklaagte',
 'Woodhill',
 'La Lucia',
 'Dinokana',
 'Peacetown',
 'Kabega',
 'Matshana',
 'Westridge',
 'Driekoppies']

### Location Details

In [47]:
location = 'Dinokana'

aid_select = ids_yr.loc[(ids_yr.LocName==location)&(ids_yr.AnswerID!=0), 'AnswerID']
sd_yr = sd[sd.AnswerID.isin(aid_select)]
sd_yr.describe()

,years_electrified,monthly_income,fridge_freezer,geyser,heater,hotplate,iron,kettle,microwave,3_plate,4_plate,tv,washing_machine,main switch,floor area,rooms,wall
count,68.000000,68.000000,35.0,0.0,7.0,28.0,33.0,35.0,4.0,3.0,8.0,43.0,3.0,68.000000,68.000000,68.000000,68.000000
mean,3.176471,770.588235,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,22.941176,66.602941,4.294118,6.514706
std,1.827985,921.204407,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.517764,39.467780,2.102300,3.605521
min,1.000000,0.000000,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,20.000000,8.000000,1.000000,1.000000
25%,2.000000,75.000000,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,20.000000,29.000000,2.000000,4.000000
50%,2.000000,780.000000,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,20.000000,56.000000,4.000000,6.000000
75%,3.250000,850.000000,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,20.000000,99.500000,6.000000,11.000000
max,7.000000,6000.000000,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,60.000000,162.000000,8.000000,11.000000


#### Electricity Supply

In [22]:
sd_yr[['years_electrified','main switch']].describe()

,years_electrified,main switch
count,68.000000,68.000000
mean,3.176471,22.941176
std,1.827985,10.517764
min,1.000000,20.000000
25%,2.000000,20.000000
50%,2.000000,20.000000
75%,3.250000,20.000000
max,7.000000,60.000000


#### Appliance Ownership

In [43]:
pd.DataFrame(sd_yr.iloc[:,4:4+len(appliances)].count()/len(sd_yr), columns=['Ownership (%hh)'])

,Ownership (%hh)
fridge_freezer,0.514706
geyser,0.000000
heater,0.102941
hotplate,0.411765
iron,0.485294
kettle,0.514706
microwave,0.058824
3_plate,0.044118
4_plate,0.117647
tv,0.632353


#### Size of Housing

In [49]:
sd_yr[['floor area','rooms']].describe()

,floor area,rooms
count,68.000000,68.000000
mean,66.602941,4.294118
std,39.467780,2.102300
min,8.000000,1.000000
25%,29.000000,2.000000
50%,56.000000,4.000000
75%,99.500000,6.000000
max,162.000000,8.000000
